In [1]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [3]:
sns.set()
input_file = '../data/japan/wforms_GAN_input_v20220805.h5'

In [4]:
f = h5py.File(input_file,'r')
f.keys()

<KeysViewHDF5 ['hypocentral_distance', 'hypocentre_depth', 'hypocentre_latitude', 'hypocentre_longitude', 'is_shallow_crustal', 'log10snr', 'magnitude', 'time_vector', 'vs30', 'waveforms']>

In [11]:
for el in [f.keys(), f.values(), f.items(), f.attrs]:
    for e in el:
        print(e)

hypocentral_distance
hypocentre_depth
hypocentre_latitude
hypocentre_longitude
is_shallow_crustal
log10snr
magnitude
time_vector
vs30
waveforms
<HDF5 dataset "hypocentral_distance": shape (1, 212598), type "<f8">
<HDF5 dataset "hypocentre_depth": shape (1, 212598), type "<f8">
<HDF5 dataset "hypocentre_latitude": shape (1, 212598), type "<f8">
<HDF5 dataset "hypocentre_longitude": shape (1, 212598), type "<f8">
<HDF5 dataset "is_shallow_crustal": shape (1, 212598), type "<f8">
<HDF5 dataset "log10snr": shape (1, 212598), type "<f8">
<HDF5 dataset "magnitude": shape (1, 212598), type "<f8">
<HDF5 dataset "time_vector": shape (5501,), type "<f8">
<HDF5 dataset "vs30": shape (1, 212598), type "<f8">
<HDF5 dataset "waveforms": shape (3, 5501, 212598), type "<f8">
('hypocentral_distance', <HDF5 dataset "hypocentral_distance": shape (1, 212598), type "<f8">)
('hypocentre_depth', <HDF5 dataset "hypocentre_depth": shape (1, 212598), type "<f8">)
('hypocentre_latitude', <HDF5 dataset "hypocentr

In [12]:
for attr in f.attrs:
    print(attr)

HP_filter_freq_lo
HP_filter_name
HP_filter_order
HP_filter_type
component_order
creation_date
h5Dir
hypo_dist_max
hypo_dist_min
magnitude_max
magnitude_min
matDir
stationList


In [26]:
def prt_attr(attr_name):
    # print(f.attrs[attr_name].shape)
    print(f.attrs[attr_name])


for attr in f.attrs:
    print(attr, end=': ')
    prt_attr(attr)


HP_filter_freq_lo: [1.]
HP_filter_name: Butterworth
HP_filter_order: [2.]
HP_filter_type: causal
component_order: NEZ
creation_date: 06-Aug-2022 00:14:53
h5Dir: /scratch/memeier/data/general/japan/bosai22/dl20220725/arx20220730/proj/wfGAN/h5/new
hypo_dist_max: [180.]
hypo_dist_min: [0.]
magnitude_max: [10.]
magnitude_min: [4.5]
matDir: /scratch/memeier/data/general/japan/bosai22/dl20220725/arx20220730/proj/wfGAN/out
stationList: dat/site/sitepub_all_en.txt


In [ ]:
magnitude_df = pd.DataFrame(f['magnitude'][0], columns=['magnitude'])

In [ ]:
magnitude_df.describe()

In [ ]:
sns.histplot(magnitude_df)

In [ ]:
np.unique(magnitude_df)

In [ ]:
dist_df = pd.DataFrame(f['hypocentral_distance'][0], columns=['hypocenter_dist'])
# print(dist_df)

combined_df = pd.concat([dist_df, magnitude_df], axis=1)
combined_df

In [ ]:
time_df = pd.DataFrame(f['time_vector'])
time_df

In [ ]:
wfs_0 = pd.DataFrame(wfs[:, 0, :])
for i in range(2):
    wfs_0.iloc[i].plot()
    plt.show()

In [ ]:
for val in np.unique(combined_df['magnitude']):
    df = combined_df[combined_df['magnitude'] == val].sort_values('hypocenter_dist')
    df.reset_index(inplace=True, drop=True)
    # print(df)
    df['hypocenter_dist'].plot()
    plt.ylim(0, 180)
    plt.show()
    

In [ ]:
mm_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))

In [ ]:
combined_df

In [ ]:
for val in np.unique(combined_df['magnitude']):
    single_quake_df = combined_df[combined_df['magnitude'] == val].sort_values('hypocenter_dist')
    single_quake_df.reset_index(inplace=True)

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    ds_val = 1
    for i in single_quake_df['index'][::ds_val]:
        dist = single_quake_df[single_quake_df['index'] == i]['hypocenter_dist'].iloc[0]
        ax.plot(time_df, wfs[i, 0]+dist, color='k', lw=0.5)

    plt.ylabel('Distance [km]')
    plt.xlabel('Time [s]')
    plt.ylim(0, 180)
    plt.show()